# UDFs and Partitioning

## Prerrequisites

Install Spark and Java in VM

In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz

In [2]:
ls -l # check the .tgz is there

total 535360
drwxr-xr-x  1 root      root      4096 Dec  8 14:36 sample_data/
drwxr-xr-x 13 110302528 1000      4096 Oct 15 09:41 spark-3.3.1-bin-hadoop2/
-rw-r--r--  1 root      root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz
-rw-r--r--  1 root      root 274099817 Oct 15 10:53 spark-3.3.1-bin-hadoop2.tgz.1


In [3]:
# unzip it
!tar xf spark-3.3.1-bin-hadoop2.tgz

In [4]:
!pip install -q findspark

Defining the environment

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [6]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Window Partitioning Exercises") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.3.1'

In [7]:
spark

In [8]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [9]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [10]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/species.csv -P /dataset
!ls /dataset

bank.csv    characters.csv    planets.csv    species.csv    vehicles.csv
bank.csv.1  characters.csv.1  planets.csv.1  species.csv.1  vehicles.csv.1


## UDFs and Partitioning Exercises

1. Read other dataset and try the repartition/colaesce
2. Try one udf function of your choice among the Df you have read

In [34]:
bank_df = spark.read.option("inferSchema",True).option("header","true").csv("/dataset/bank.csv",sep =';')

bank_df_part = bank_df.repartition(500)

bank_df_coal = bank_df_part.coalesce(2)

bank_df_part.withColumn("id", spark_partition_id()).groupBy("id").count().orderBy(col("id")).show()

bank_df_coal.withColumn("id", spark_partition_id()).groupBy("id").count().orderBy(col("id")).show()







bank_df_part.show()
bank_df_part.withColumn("id", spark_partition_id()).where(col("id")==320).show()

multiply5 = udf(lambda z: z*5)


bank_df.select(col("job"), \
    multiply5(col("balance"))) \
   .show()




+---+-----+
| id|count|
+---+-----+
|  0|    9|
|  1|    9|
|  2|    9|
|  3|    9|
|  4|    9|
|  5|    9|
|  6|    9|
|  7|    9|
|  8|    9|
|  9|    9|
| 10|    9|
| 11|    9|
| 12|    9|
| 13|    9|
| 14|    9|
| 15|    9|
| 16|    9|
| 17|    9|
| 18|    9|
| 19|    9|
+---+-----+
only showing top 20 rows

+---+-----+
| id|count|
+---+-----+
|  0| 2260|
|  1| 2261|
+---+-----+

+---+-----------+--------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|        job| marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+-----------+--------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 31|     admin.|  single|secondary|     no|    757|     no|  no|cellular|  3|  feb|     343|       2|   -1|       0| unknown| no|
| 50|  housemaid| married|secondary|     no|   7317|    yes|  no|cellular| 13|  may|     